In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', 80)

In [3]:
# Read data
branded_data = pd.read_csv('../data/raw/FoodData_Central_csv_2023-04-20/branded_food.csv')
nutrient_data = pd.read_csv('../data/raw/FoodData_Central_csv_2023-04-20/food_nutrient.csv')

/tmp/ipykernel_1150/574112235.py:2: DtypeWarning: Columns (2,3,4,6,9,12,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  branded_data = pd.read_csv('../data/raw/FoodData_Central_csv_2023-04-20/branded_food.csv')
/tmp/ipykernel_1150/574112235.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  nutrient_data = pd.read_csv('../data/raw/FoodData_Central_csv_2023-04-20/food_nutrient.csv')


In [13]:
nutrient_data

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired
0,13706913,1105904,1003,0.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN
1,13706914,1105904,1004,93.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN
2,13706915,1105904,1005,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN
3,13706924,1105904,1110,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN
4,13706918,1105904,1079,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
24743721,31781973,2554914,1110,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN
24743722,31781965,2554914,1005,13.33,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN
24743723,31781974,2554914,1253,0.00,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN
24743724,31781963,2554914,1003,23.33,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
nutrient_info = pd.read_csv('../data/raw/FoodData_Central_csv_2023-04-20/nutrient.csv')

In [5]:
# We want to use the ingredients for the model,
# so we discard records that dont contain ingredients. 
branded_data = branded_data[branded_data.ingredients.notna()]

# We are getting all the information that has nutrient information 
branded_data = branded_data[branded_data.fdc_id.isin(nutrient_data.fdc_id)]

# We want only the same products ('gtin_upc' makes the deal)
clean_gtin_upc = branded_data.gtin_upc.astype('string')\
.str.replace('-', '')\
.str.replace(' ', '')\
.str.replace('X', '')\
.str.replace('>', '')\
.str.replace('`', '')\
.str.replace('.', '')
gtin_upc_conditions = clean_gtin_upc.str.isdigit()

branded_data['clean_gtin_upc'] = clean_gtin_upc

branded_data = branded_data[gtin_upc_conditions]

# Lets sort the dataframe 
branded_data['available_date'] = pd.to_datetime(branded_data['available_date'])
branded_data.sort_values('available_date', inplace=True)
branded_data_reduced = branded_data.groupby('clean_gtin_upc').last().reset_index()


In [12]:
branded_data_reduced[['fdc_id', 'ingredients', 'branded_food_category']]

,fdc_id,ingredients,branded_food_category
0,555505,"ORGANIC BLUE CORN, ORGANIC SUNFLOWER OIL, SEA SALT, TRACE OF LIME.","Chips, Pretzels & Snacks"
1,1055419,"ROASTED PEANUTS (PEANUTS, PEANUT OR CANOLA OIL, SALT), SESAME STICKS (UNBLEA...",Other Snacks
2,1055431,"YOGURT RAISINS, TAMARI ROASTED ALMONDS, ORGANIC TAMARI ROASTED SOY NUTS, DAR...","Popcorn, Peanuts, Seeds & Related Snacks"
3,1055439,"ORGANIC ROLLED OATS, ORGANIC EVAPORATED CANE JUICE, ORGANIC QUINOA FLAKES, O...",Cereal
4,1055441,"DRY ROASTED ALMONDS, HATCH GREEN CHILE SEASONING (ORGANIC CHEDDAR CHEESE POW...","Popcorn, Peanuts, Seeds & Related Snacks"
...,...,...,...
564439,555751,"PORK, WATER, SALT, DEXTROSE, SODIUM LACTATE, SODIUM PHOSPHATES, DEHYDRATED P...",Cooked & Prepared
564440,555737,"PORK, WATER, SALT, DEXTROSE, SODIUM LACTATE, SODIUM PHOSPHATES, DEHYDRATED P...","Pepperoni, Salami & Cold Cuts"
564441,1129231,"SUGAR, BUTTER(MILK), WHEAT FLOUR(WHEAT FLOUR, MALTED BARLEY FLOUR), CHOCOLAT...",Cookies & Biscuits
564442,2067237,"TOMATO PASTE, DISTILLED VINEGAR, WATER, HIGH FRUCTOSE CORN SYRUP, HORSERADIS...","Oriental, Mexican & Ethnic Sauces"


In [9]:
#branded_data[branded_data.clean_gtin_upc=='000000000000']
nutrient_data[nutrient_data.fdc_id==555505].merge(nutrient_info, left_on='nutrient_id', right_on='id')

,id_x,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,id_y,name,unit_name,nutrient_nbr,rank
0,4320669,555505,1008,500.00,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1008,Energy,KCAL,208.0,300.0
1,4320673,555505,1257,0.00,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1257,"Fatty acids, total trans",G,605.0,15400.0
2,4320667,555505,1004,25.00,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1004,Total lipid (fat),G,204.0,800.0
3,4320668,555505,1005,67.86,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1005,"Carbohydrate, by difference",G,205.0,1110.0
4,4320674,555505,1258,3.57,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1258,"Fatty acids, total saturated",G,606.0,9700.0
5,4320675,555505,1292,5.36,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1292,"Fatty acids, total monounsaturated",G,645.0,11400.0
6,4320676,555505,1293,10.71,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1293,"Fatty acids, total polyunsaturated",G,646.0,12900.0
7,4320672,555505,1093,286.00,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1093,"Sodium, Na",MG,307.0,5800.0
8,4320670,555505,2000,0.00,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,2000,"Sugars, total including NLEA",G,269.0,1510.0
9,4320671,555505,1079,3.60,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,1079,"Fiber, total dietary",G,291.0,1200.0


In [16]:
nutrient_info

,id,name,unit_name,nutrient_nbr,rank
0,2047,Energy (Atwater General Factors),KCAL,957.0,280.0
1,2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
2,1001,Solids,G,201.0,200.0
3,1002,Nitrogen,G,202.0,500.0
4,1003,Protein,G,203.0,600.0
...,...,...,...,...,...
469,2061,"Ergosta-7,22-dienol",MG,NaN,16211.0
470,2062,"Ergosta-5,7-dienol",MG,NaN,16211.0
471,2063,Verbascose,G,NaN,2450.0
472,2064,Oligosaccharides,MG,NaN,2250.0


In [34]:
branded_data['ingredients_manipulation'] = branded_data.ingredients
branded_data['ingredients_manipulation'] = branded_data.ingredients_manipulation.str.lower().str.replace('ingredients:', '')
branded_data['ingredients_manipulation'] = branded_data.ingredients_manipulation.str.replace('made of:', '').str.replace('*', '').str.replace('.', '')
branded_data['semicolon_count'] = branded_data.ingredients_manipulation.str.count(':')

In [36]:
branded_data

,fdc_id,brand_name,gtin_upc,branded_food_category,ingredients,ingredients_manipulation,semicolon_count
0,1105904,NaN,27000612323,Oils Edible,Vegetable Oil,vegetable oil,0
1,1105905,NaN,51000198808,Herbs/Spices/Extracts,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2% OF: MIREPOIX (CARROTS, CELERY, ONIONS), SALT, NATURAL FLAVORING, YEAST EXTRACT, CANE SUGAR.","beef stock, contains less than 2% of: mirepoix (carrots, celery, onions), salt, natural flavoring, yeast extract, cane sugar",1
2,1105906,NaN,51000213273,Prepared Soups,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREAM, VEGETABLE OIL (CORN, CANOLA, AND/OR SOYBEAN), CELERY, CONTAINS LESS THAN 2% OF: MODIFIED FOOD STARCH, SALT, WHEAT FLOUR, SOY PROTEIN CONCENTRATE, UNCURED SMOKED BACON PIECES-NO NITRATES OR NITRITES ADDED, EXCEPT FOR THOSE NATURALLY OCCURRING IN SEA SALT (PORK, WATER, SEA SALT, SUGAR), DRIED ONIONS, SPICES, SODIUM PHOSPHATE, FLAVORING, CLAM EXTRACT, SUCCINIC ACID, SUGAR, SOY LECITHIN, SOY SAUCE (SOYBEANS, WHEAT, SALT).CONTAINS: COD, WHEAT, MILK, SOY.","clam stock, potatoes, clams, cream, vegetable oil (corn, canola, and/or soybean), celery, contains less than 2% of: modified food starch, salt, wheat flour, soy protein concentrate, uncured smoked bacon pieces-no nitrates or nitrites added, except for those naturally occurring in sea salt (pork, water, sea salt, sugar), dried onions, spices, sodium phosphate, flavoring, clam extract, succinic acid, sugar, soy lecithin, soy sauce (soybeans, wheat, salt)contains: cod, wheat, milk, soy",2
3,1105907,NaN,51000213303,Prepared Soups,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, VEGETABLE OIL (CORN, CANOLA, AND/OR SOYBEAN), MODIFIED FOOD STARCH, CHEDDAR CHEESE** (CHEDDAR CHEESE [CULTURED MILK, SALT, ENZYMES], WHEY, SALT, SODIUM PHOSPHATE), CONTAINS LESS THAN 2% OF: BUTTER, PARMESAN AND CHEDDAR CHEESE (MILK, CULTURES, SALT, ENZYMES), WHEAT FLOUR, SALT, POTATOES**, ONIONS**, SOY PROTEIN CONCENTRATE, ROASTED GARLIC**, ANNATTO EXTRACT FOR COLOR, SOY LECITHIN. **DRIEDCONTAINS: WHEAT, MILK, SOY.","water, cream, broccoli, celery, vegetable oil (corn, canola, and/or soybean), modified food starch, cheddar cheese (cheddar cheese [cultured milk, salt, enzymes], whey, salt, sodium phosphate), contains less than 2% of: butter, parmesan and cheddar cheese (milk, cultures, salt, enzymes), wheat flour, salt, potatoes, onions, soy protein concentrate, roasted garlic, annatto extract for color, soy lecithin driedcontains: wheat, milk, soy",2
4,1105908,NaN,51000224637,Herbs/Spices/Extracts,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN 2% OF: YEAST EXTRACT, DEHYDRATED CHICKEN, NATURAL FLAVORING, CARROT JUICE CONCENTRATE, CELERIAC JUICE CONCENTRATE, CHICKEN FAT, ONION EXTRACT.","chicken stock, contains less than 2% of: yeast extract, dehydrated chicken, natural flavoring, carrot juice concentrate, celeriac juice concentrate, chicken fat, onion extract",1
...,...,...,...,...,...,...,...
1845292,2554910,LIFESTYLEFOODS,898425002682,"Pickles, Olives, Peppers & Relishes","ARCADIAN HARVEST LETTUCE (BLEND OF LEAF LETTUCES), RANCH DRESSING (BUTTERMILK [CULTURED PASTEURIZED PART-SKIM MILK, NONFAT MILK POWDER, SALT], SOYBEAN OIL AND/OR CANOLA OIL, WATER, EGG YOLK, NATURAL FLAVORS, CONTAINS LESS THAN 2% OF: DEHYDRATED GARLIC, DEHYDRATED ONION, DEHYDRATED PARSLEY, DISTILLED VINEGAR, GLUCONO DELTA LACTONE, PROPYLENE GLYCOL ALGINATE, SALT, SODIUM CITRATE, SPICES [CONTAINS MUSTARD], SUGAR, TITANIUM DIOXIDE [COLOR], XANTHAN GUM), TOMATOES, RED CABBAGE, HAM WITH WATER AND SMOKE FLAVOR ADDED (PORK [CURED WITH: WATER, DEXTROSE, SALT, SODIUM PHOSPHATE, SODIUM ERYTHORBATE, NATURAL SMOKE FLAVOR, SODIUM NITRITE]), WHITE TURKEY (WHITE TURKEY, WATER, MODIFIED FOOD STARCH, SALT, DEXTROSE, SODIUM PHOSPHATE), CHEDDAR CHEESE (PASTEURIZED MILK, CHEESE CULTURES, SALT, ENZYMES, ANNATTO, VEGETABLE COLOR [IF COLORED], CORN STARCH [ANTI-CAKING AGENT]).","arcadian harvest lettuce (blend of leaf lettuces), ranch dressing (buttermilk [cultured pasteurized part-skim milk, no

In [9]:
print(f'There are {branded_data.branded_food_category.nunique()} categories in this dataset')

There are 428 categories in this dataset


In [35]:
branded_data.branded_food_category.value_counts().head(30)

branded_food_category
Popcorn, Peanuts, Seeds & Related Snacks           87555
Candy                                              83980
Cheese                                             76177
Ice Cream & Frozen Yogurt                          58400
Cookies & Biscuits                                 50611
Chips, Pretzels & Snacks                           47527
Breads & Buns                                      45169
Pickles, Olives, Peppers & Relishes                42293
Fruit & Vegetable Juice, Nectars & Fruit Drinks    39468
Chocolate                                          37835
Snack, Energy & Granola Bars                       37812
Cakes, Cupcakes, Snack Cakes                       36323
Cereal                                             35036
Yogurt                                             34718
Seasoning Mixes, Salts, Marinades & Tenderizers    32253
Soda                                               31713
Other Snacks                                       29571
Pasta by 

In [18]:
# We have MAAANY Bread and Buns

In [11]:

def get_similarities_of_dataset(data, category, sample_size, id_col='fdc_id', random_state=42):
    """
    """
    sample_data = data[category].copy()[[id_col]].sample(sample_size,random_state=random_state)
    sample_data[f'{id_col}2'] = [sample_data[id_col].to_list()]*len(sample_data)
    sample_data_result = sample_data.explode(f'{id_col}2', ignore_index=True)
    sample_data_result['similarity'] = 1
    sample_data.reset_index(drop=True, inplace=True)
    return sample_data, sample_data_result

def get_differences_between_datasets(data0, data1, id_col):
    data_a = pd.concat([data0.drop(f'{id_col}2', axis=1), data1.drop(id_col, axis=1)], axis=1)
    data_b = pd.concat([data1.drop(f'{id_col}2', axis=1), data0.drop(id_col, axis=1)], axis=1)
    data = pd.concat([data_a, data_b])
    data_res = data.explode(f'{id_col}2', ignore_index=True)
    data_res['similarity'] = 0
    return data_res
    

def get_simple_dataset(food_dict, sample_size=100, categ1=None, categ2=None):
    # Similarities for category 1 (asume everything in category is the same):
    food0, food0_res = get_similarities_of_dataset(food_dict, categ1, sample_size=sample_size, id_col='fdc_id')
    # Similarities for category 2 (asume everything in category is the same):
    food1, food1_res = get_similarities_of_dataset(food_dict, categ2, sample_size=sample_size, id_col='fdc_id')
    # Differences for categories: (asume everything outside category is not the same):
    food2_res = get_differences_between_datasets(food0, food1, id_col='fdc_id')
    # Merge results
    results = pd.concat([food0_res, food1_res, food2_res], axis=0)
    return results

In [12]:
# First we need to create a dictionary for this functions to work.
food_dict = {}
food_dict['bread'] = branded_data[branded_data.branded_food_category == 'Breads & Buns']
food_dict['yogurt'] = branded_data[branded_data.branded_food_category == 'Yogurt']
food_dict['soda'] = branded_data[branded_data.branded_food_category == 'Soda']
food_dict['cookies'] = branded_data[branded_data.branded_food_category == 'Cookies & Biscuits']
food_dict['cereal'] = branded_data[branded_data.branded_food_category == 'Cereal']

In [28]:
test_df = pd.concat([
    get_simple_dataset(food_dict, 10, 'bread', 'yogurt'),
    get_simple_dataset(food_dict, 10, 'bread', 'soda'),
    get_simple_dataset(food_dict, 10, 'bread', 'cookies'),
    get_simple_dataset(food_dict, 10, 'yogurt', 'cookies'),
    get_simple_dataset(food_dict, 10, 'soda', 'yogurt'),

]).drop_duplicates()

In [29]:
test_df

,fdc_id,fdc_id2,similarity
0,2037398,2037398,1
1,2037398,1557736,1
2,2037398,2374051,1
3,2037398,2138331,1
4,2037398,1249786,1
...,...,...,...
195,544139,2335225,0
196,544139,1967840,0
197,544139,498542,0
198,544139,1363363,0


In [30]:
test_df[test_df.fdc_id==2530463]

,fdc_id,fdc_id2,similarity
10,2530463,793289,1
11,2530463,2530463,1
12,2530463,1684750,1
13,2530463,1325638,1
14,2530463,2191414,1
15,2530463,873892,1
16,2530463,2292401,1
17,2530463,1696147,1
18,2530463,1706760,1
19,2530463,544139,1


In [31]:
branded_data[branded_data.fdc_id==2530463].ingredients_manipulation.str.split(',')

1820845    [strawberry - cultured pasteurized grade a lowfat milk,  sugar,  strawberries,  modified cornstarch,  nonfat milk solids,  pectin,  natural flavors,  citric acid,  black carrot juice (color),  vitamin a palmitate,  vitamin d3, peach - cultured pasteurized grade a lowfat milk,  sugar,  peaches,  modified cornstarch,  nonfat milk solids,  pectin,  natural flavors,  citric acid,  annatto (color),  vitamin a palmitate,  vitamin d3]
Name: ingredients_manipulation, dtype: object

In [32]:
branded_data[branded_data.fdc_id==2083392].ingredients_manipulation.str.split(',')

1422943    [carbonated water,  high fructose corn syrup,  citric acid,  natural and artificial flavors,  fd&c yellow #6,  caramel color,  sodium benzoate.]
Name: ingredients_manipulation, dtype: object

In [33]:
branded_data[branded_data.fdc_id==1696147].ingredients_manipulation.str.split(',')

1089972    [lowfat yogurt (cultured pasteurized nonfat milk,  cream),  cane sugar,  water,  hazelnuts,  wheat flour,  wheat starch,  sunflower oil,  natural flavors,  milk,  milk protein,  coffee powder,  fruit pectin,  chocolate,  cocoa butter,  locust bean gum,  guar gum,  cinnamon,  lemon juice concentrate,  soy lecithin,  vanilla extract,  salt.]
Name: ingredients_manipulation, dtype: object

The problem here is that the ingredients are not-structured which might be complicated. Also, there might no be information on the ingredients. One should look at the survey foods so gain all of this information. 